In [1]:
import numpy as np
import cv2 as cv
import os
import faiss

In [2]:
def get_sift_keypoints(img):
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create(nfeatures=5000)
    kp, des = sift.detectAndCompute(gray, None)
    return kp, des

In [3]:
labels = []
features = []
for image_name in os.listdir('images'):
    building_name = image_name[0:image_name.find('.')]
    
    image_path = 'images/' + image_name
    img = cv.imread(image_path)
    
    kp, des = get_sift_keypoints(img)
    labels += [building_name for i in range(len(kp))]
    
    features.append(np.vstack(des))
    
    # img=cv.drawKeypoints(gray,kp,img)
    # cv.imshow('image',img)
    # cv.waitKey(0)

In [4]:
len(labels)

14325

In [5]:
features_ar = np.vstack(features)
features_ar.shape

(14325, 128)

In [6]:
index = faiss.IndexFlatL2(128)
index.add(features_ar)
print(index.ntotal)

14325


In [7]:
def find_closest_image_match(img):
    return None

In [12]:
search_image_path = 'search.jpg'
img = cv.imread(search_image_path)
kp, des = get_sift_keypoints(img)
k = 1

In [13]:
preds = {}
for d in des:
    D, I = index.search(d.reshape((1, 128)), k)
    pred = labels[I[0, 0]]
    if pred not in preds:
        preds[pred] = 0
    preds[pred] += 1

preds

{'kellogg': 970, 'tech': 489, 'kellogg_full': 855, 'segal': 382}